
(tutorials-pockels)=

# Clamped Pockels tensor of AlAs

In this tutorial you will learn how to calculate the clamped Pockels tensor using `aiida-vibroscopy`.

For the demonstration, we will make use of AlAs, a non-centrosymmetric semiconductor material.

Let's get started!

## Pockels tensor formalism

The Pockels effect in a linear electro-optic effect relating an external electric field $\mathcal{E}_k$ modulation to a change in refractive index $n_{ij}$ in a non-centrosymmetric material. The Pockels tensor $r_{ijk}$ is defined within the formalism of the index ellipsoid as

\begin{equation}
    \Delta \Bigg(\frac{1}{n_{ij}^2} \Bigg)
    =
    \sum_{k=1}^3 r_{ijk} \mathcal{E}_k
\end{equation}

where indices $i$, $j$, and $k$ represent the polarizations (in a Cartesian basis) of the incoming optical field, the outgoing optical field, and the modulating electrical field, respectively.

The Pockels tensor is a function of electric field, atomic positions, and lattice strains. Vibroscopy computes the clamped contribution to $r_{ijk}$ only, thus neglecting the lattice strains contributions. This approximation is valid for electric field modulations that exceed lattice resonance frequencies. In such case, $r_{ijk}$ can be written as $r_{ijk}^{el} + r_{ijk}^{ion}$, i.e., a __high-frequency__ (electronic) and a __phonon contribution__ (ionic)

\begin{equation}
    r_{ijk}^{el} + r_{ijk}^{ion} 
    = 
    -\frac{2}{n_{ii}^2 n_{jj}^2} \chi_{ijk}^{(2)} 
    - \frac{1}{n_{ii}^2 n_{jj}^2} \sum_m \frac{\alpha_{ij}^m p_{m,k}}{\omega_m^2}
\end{equation}

where $\chi_{ijk}^{(2)}$ is the DC-field second-order dielectric susceptibility. The $\alpha_{ij}^m$ variable is the Raman susceptibility tensor associated with phonon mode $m$. It is calculated as 

\begin{equation}
    \alpha_{ij}^m
    =
    \sum_{\kappa, \beta} \frac{\partial \chi_{ij}^{(1)}}{\partial \tau_{\kappa, \beta}} u_m(\kappa, \beta)
\end{equation}

where $\frac{\partial \chi_{ij}^{(1)}}{\partial \tau_{\kappa, \beta}}$ is the Raman tensor introduced in [the tutorial on vibrational spectra](3_iraman) and $u_m(\kappa, \beta)$ is the eigenvector associated with atom $\kappa$ and polarization direction $\beta$. The mode polarity $p_{m, k}$ is calculated as 

\begin{equation}
    p_{m,k}
    =
    \sum_{\kappa, \beta} Z_{\kappa, k \beta}^* u_m(\kappa, \beta)
\end{equation}

with the Born effective charges $Z_{\kappa, k \beta}^*$ introduced in [the tutorial on dielectric tensor calculations](2_dielectric).

::: {note}
The Pockels formalism was originally developed in the late 60s, however its implementation in Vibroscopy is based on the successive work of Veithen \textit{et al.}. See:

- Johnston Jr, W.D., Phys. Rev. B, __1.8__, 3494 (__1970__)
- Veithen, M., Xavier Gonze, and Ph Ghosez, Phys. Rev. B, __71.12__, 125107 (__2005__)
:::

Attention! The __non-analytical constants__ (NAC) should be accounted for whenever polar materials are investigated (refer to [the tutorial on phonon and dielectric properties of AlAs](4_polar)).

In [1]:
from local_module import load_temp_profile

# If you download this file, you can run it with your own profile.
# Put these lines instead:
# from aiida import load_profile
# load_profile()
data = load_temp_profile(
    name="clamped-pockels-tutorial",
    add_computer=True,
    add_pw_code=True,
    add_sssp=True,
)

Report: downloading patch versions information... 

/opt/conda/lib/python3.9/site-packages/aiida/orm/nodes/data/code/legacy.py:46: AiidaDeprecationWarning: The `Code` class is deprecated. To create an instance, use the `aiida.orm.nodes.data.code.installed.InstalledCode` or `aiida.orm.nodes.data.code.portable.PortableCode` for a "remote" or "local" code, respectively. If you are using this class to compare type, e.g. in `isinstance`, use `aiida.orm.nodes.data.code.abstract.AbstractCode`. (this will be removed in v3)
  warn_deprecation(
/opt/conda/lib/python3.9/site-packages/aiida/orm/nodes/data/code/legacy.py:58: AiidaDeprecationWarning: The `Code` plugin is deprecated, use the `InstalledCode` (`core.code.remote`) instead. (this will be removed in v3)
  warn_deprecation(
/opt/conda/lib/python3.9/site-packages/aiida/orm/nodes/data/code/legacy.py:493: AiidaDeprecationWarning: `Code.set_remote_computer_exec` method is deprecated, use `InstalledCode`. (this will be removed in v3)
  warn_deprecation('`Code.set_remote_computer_exec` method is 

 [OK]
Report: downloading selected pseudopotentials archive...  [OK]
Report: downloading selected pseudopotentials metadata...  [OK]


## Importing the structure from COD

Let's import the AlAs structure from the COD database via AiiDA.

In [2]:
from struct import Struct
from aiida.plugins import DbImporterFactory
from aiida.plugins import DataFactory

CodDbImporter = DbImporterFactory('cod')

cod = CodDbImporter()
results = cod.query(id='1540257') # AlAs 1540257
structure = results[0].get_aiida_structure() # it has 8 atoms

/opt/conda/lib/python3.9/site-packages/aiida/plugins/entry_point.py:338: AiidaDeprecationWarning: The entry point `cod` is deprecated. Please replace it with `core.cod`. (this will be removed in v3)
  warn_deprecation(f'The entry point `{name}` is deprecated. Please replace it with `core.{name}`.', version=3)
/opt/conda/lib/python3.9/site-packages/aiida/tools/data/cif.py:134: FutureWarning: get_structures is deprecated
get_structures is deprecated and will be removed in 2024. Use parse_structures instead.The only difference is that primitive defaults to False in the new parse_structures method.So parse_structures(primitive=True) is equivalent to the old behavior of get_structures().
  structures = parser.get_structures(**parameters)



## Running the `IRamanSpectraWorkChain`

Let's import the WorkChain and run it!

As usual, we use the `get_builder_from_protocol` to get a __prefilled__ builder with __all inputs__. These inputs should be considered __not as converged parameters__, but as a good starting point. You may also need to tweak some parameters, e.g. add magnetization etc., depending on your case. 

In [ ]:
from aiida.plugins import WorkflowFactory
from aiida.engine import run_get_node
from aiida_quantumespresso.common.types import ElectronicType

# NOTE: it is the same workflow as for computing IR and Raman spectra!
PockelsWorkChain = WorkflowFactory("vibroscopy.spectra.iraman")

# To make the calculations run faster
scf_overrides = {'scf':{'pw':{'parameters':{'SYSTEM':{'ecutwfc':30, 'ecutrho':30*8}}}}}

builder = PockelsWorkChain.get_builder_from_protocol(
    code=data.pw_code,
    structure=structure,
    protocol="fast",
    overrides={'dielectric':scf_overrides, 'phonon':scf_overrides},
    kwargs={'electronic_type' :ElectronicType.INSULATOR},
)

results, calc = run_get_node(builder)

03/06/2025 09:40:24 AM <15160> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [124|IRamanSpectraWorkChain|run_spectra]: submitting `HarmonicWorkChain` <PK=126>
03/06/2025 09:40:24 AM <15160> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [126|HarmonicWorkChain|run_phonon]: submitting `PhononWorkChain` <PK=129>
03/06/2025 09:40:25 AM <15160> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [126|HarmonicWorkChain|run_dielectric]: submitting `DielectricWorkChain` <PK=130>
03/06/2025 09:40:26 AM <15160> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [130|DielectricWorkChain|run_base_scf]: launching base scf PwBaseWorkChain<143>
03/06/2025 09:40:26 AM <15160> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [143|PwBaseWorkChain|run_process]: launching PwCalculation<146> iteration #1
03/06/2025 09:40:27 AM <15160> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [129|PhononWorkCh

## Calculating the clamped Pockels tensor

Let's print the clamped Pockels tensor `r_tot`, its electronic contribution `r_el`, and its ionic contribution `r_ion`.

In [5]:
vibro = calc.outputs.vibrational_data.numerical_accuracy_4
r_tot, r_el, r_ion = vibro.run_clamped_pockels_tensor()
print('Clamped Pockels tensor (r_tot) of AlAs in pm/V: \n', r_tot.round(2), '\n')
print('Electronic contribution to r_tot in pm/V: \n', r_el.round(2), '\n')
print('Ionic contribution to r_tot in pm/V: \n', r_ion.round(2), '\n')

Clamped Pockels tensor (r_tot) of AlAs in pm/V: 
 [[[ 0.    0.   -0.  ]
  [-0.    0.   -0.75]
  [-0.   -0.75 -0.  ]]

 [[-0.    0.   -0.75]
  [-0.   -0.    0.  ]
  [-0.75  0.   -0.  ]]

 [[-0.   -0.75 -0.  ]
  [-0.75  0.   -0.  ]
  [-0.   -0.   -0.  ]]] 

Electronic contribution to r_tot in pm/V: 
 [[[-0.   -0.   -0.  ]
  [-0.   -0.   -1.22]
  [-0.   -1.22 -0.  ]]

 [[-0.   -0.   -1.22]
  [-0.    0.    0.  ]
  [-1.22  0.   -0.  ]]

 [[-0.   -1.22 -0.  ]
  [-1.22  0.   -0.  ]
  [-0.   -0.   -0.  ]]] 

Ionic contribution to r_tot in pm/V: 
 [[[ 0.    0.    0.  ]
  [-0.    0.    0.47]
  [ 0.    0.47  0.  ]]

 [[-0.    0.    0.47]
  [ 0.   -0.   -0.  ]
  [ 0.47  0.    0.  ]]

 [[ 0.    0.47  0.  ]
  [ 0.47  0.    0.  ]
  [ 0.    0.    0.  ]]] 



Due to symmetry (AlAs belongs to space group number 216), we obtain $-0.76$ pm/V for each $r_{123} = r_{132} = r_{213} = r_{231} = r_{321} = r_{312}$ clamped Pockels coefficient. This value is very small and of negligible interest for practical opto-electronic applications - hence the lack of available experimental data, as far as we know. We chose AlAs for the sake of running this tutorial within a reasonable amount of time. However, once you get the hang of it, you can go on and study more relevant materials.

::: {warning}
In the literature, it is common to refer to Pockels coefficients using the Voigt notation, where the first two indices are contracted as follows: $11 \rightarrow 1$, $22 \rightarrow 2$, $33 \rightarrow 3$, $23 = 32 \rightarrow 4$, $31 = 13 \rightarrow 5$, and $12 = 21 \rightarrow 6$. For AlAs for instance, $r_{123} = r_{63}$, $r_{132} = r_{52}$, ...
:::

You will soon be able to find more detailed information about the clamped Pockels tensor calculation, its implementation in Vibroscopy, and a practical example on the tetragonal phase of barium titanate. We'll keep you updated about our paper release!